In [ ]:
%pip3 install gradio
%pip install opencv-python
%pip install scikit-learn
%pip install sentence-transformers
%pip install transformers
%pip install torch


In [ ]:
!pip3 freeze > requirements.txt

# handwriting reader

In [1]:
import cv2
import numpy as np
import math
%matplotlib inline
from matplotlib import pyplot as plt

In [6]:
def parametricIntersect(r1:float, t1:float, r2:float, t2:float) -> (bool, int, int): # type: ignore
    ct1:float=math.cos(t1)
    st1:float=math.sin(t1)
    ct2:float=math.cos(t2)
    st2:float=math.sin(t2)
    d=ct1*st2-st1*ct2 # determinative (rearranged matrix for inverse)
    if d!= 0:
        x=int((st2*r1-st1*r2)/d)
        y=int((-ct2*r1+ct1*r2)/d)
        return True, x, y
    else: # //lines are parallel and will NEVER intersect!
        return False, -1, -1

In [7]:
def chop_image(img_path, vertical_thresh):
    img = cv2.imread(img_path)
    # convert to black and white
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray,50,150,apertureSize = 3)
    lines = cv2.HoughLines(edges,1,np.pi/180,200)
    vertical_threshold = int(img.shape[0] * vertical_thresh)
    final_lines = []
    skip_lines = []
    if lines is None:
        return [img]
    for i in range(len(lines)):
        rho1 = lines[i][0][0]
        close_lines = []
        for j in range(i+1, len(lines)):
            rho2 = lines[j][0][0]
            if i not in skip_lines and j not in skip_lines:
                if abs(rho2 - rho1) < vertical_threshold:
                    close_lines.append(lines[i][0])
                    skip_lines.append(j)
                if len(close_lines) == 0:
                    final_lines.append(lines[i][0])
                else:
                    best_line = [math.inf, math.inf]
                    for close_line in close_lines:
                        if close_line[0] < best_line[0]:
                            best_line = close_line
                    final_lines.append(best_line)
    final_final_lines = np.unique(np.array(final_lines), axis=0)
    chopped_images = []
    for line in range(len(final_final_lines)):
        if line == 0:
            y0 = 0
        else:
            y0 = int(final_final_lines[line - 1][0])
        y1 = int(final_final_lines[line][0])
        # print(y0, y1)
        # img = cv2.imread(im_path)

        # plt.imsave(f"line{line}.png", img[y0:y1, :, :])
        chopped_images.append(img[y0:y1, :, :])
    return chopped_images

In [8]:
im_path1 = "/Users/isaac/Desktop/Creative/Coding/momApp/Beauty from Ashes Survey Data/handwriting/Screenshot 2024-04-21 at 5.09.43 PM.png"
im_path2 = "/Users/isaac/Desktop/Creative/Coding/momApp/Beauty from Ashes Survey Data/handwriting/Screenshot 2024-04-21 at 5.09.43 PM copy.png"
chopped = chop_image(im_path2, 0.1)
for i, img in enumerate(chopped):
    plt.imsave(f"line{i}.png", img)

In [88]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from PIL import Image

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [ ]:
pipe = pipeline("image-to-text", model="microsoft/trocr-base-handwritten")

In [95]:
for i, img in enumerate(chopped):
    pil_im = Image.fromarray(img)
    pil_im.save(f"line{i}.png")
    text = pipe(pil_im)[0]["generated_text"]
    print(text)

I loved this conference. The speakers
 were great. Great activities and topics.
 Great location. A little far for me, but
 so worth it


In [26]:
feedback = [
        "I loved this conference. The speakers were great, Great activities and topics. Great location. A little too far for me, but so worth it.",
        "Very encouraging, love the first message and the last",
        "I loved the overall themes and felt inspired to maintain mental health as well as stepping in to who God has called me to be",
        "Gut health, We are loved by God, Change self talk",
        "I love the fact that Jesus is the center of every speaker it is very refreshing",
        ]

conference_theme = [
    "God is doing a new thing",
    "A new thing",
    "Season of Hope",
    "Women Arise",
    "What does being a leader look like under the leadership of your husband",
    "Healthy Relationships",
    "Medical health (any) as well as emotional health",
    "More wooden bowls from zambia",
    "A lesson on losses"]

house_names = [
    "House of Hope",
    "Hope Arise",
    "Arise Hope",
    "She is my delight",
    "My delight",
    "Tirzah",
    "Encompassed by Grace",
    "Home of Grace",
    "Grace Point",
    "Hepzibah (my delight in her)",
    "Angle aware",
    "Child of God",
    "Gods Love Children"]

# app

In [102]:
# TODO
# ability to exclude certain words from process
# better handwriting detection
# fix problem with 1 or 0 line of text
# fix issue with 0 images

In [3]:
from transformers import pipeline
import cv2
import math
import gradio as gr
from PIL import Image
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from sklearn.preprocessing import normalize
from sentence_transformers import SentenceTransformer
import os
from sklearn.feature_extraction.text import TfidfVectorizer

def chop_image(img_path, vertical_thresh):
    img = cv2.imread(img_path)
    # convert to black and white
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray,50,150,apertureSize = 3)
    lines = cv2.HoughLines(edges,1,np.pi/180,200)
    vertical_threshold = int(img.shape[0] * vertical_thresh)
    final_lines = []
    skip_lines = []
    if lines is None:
        return [img]
    for i in range(len(lines)):
        rho1 = lines[i][0][0]
        close_lines = []
        for j in range(i+1, len(lines)):
            rho2 = lines[j][0][0]
            if i not in skip_lines and j not in skip_lines:
                if abs(rho2 - rho1) < vertical_threshold:
                    close_lines.append(lines[i][0])
                    skip_lines.append(j)
                if len(close_lines) == 0:
                    final_lines.append(lines[i][0])
                else:
                    best_line = [math.inf, math.inf]
                    for close_line in close_lines:
                        if close_line[0] < best_line[0]:
                            best_line = close_line
                    final_lines.append(best_line)
    final_final_lines = np.unique(np.array(final_lines), axis=0)
    chopped_images = []
    for line in range(len(final_final_lines)):
        if line == 0:
            y0 = 0
        else:
            y0 = int(final_final_lines[line - 1][0])
        y1 = int(final_final_lines[line][0])
        # print(y0, y1)
        # img = cv2.imread(im_path)

        # plt.imsave(f"line{line}.png", img[y0:y1, :, :])
        chopped_images.append(img[y0:y1, :, :])
    return chopped_images

def image_to_text(image_list):
    single_line_text_images = []
    output_text = ""
    for image in image_list:
        print(image)
        single_line_text_images += chop_image(image, 0.1)
    print(len(single_line_text_images))
    for img in single_line_text_images:
        pil_im = Image.fromarray(img)
        text = pipe(pil_im)[0]["generated_text"]
        output_text += f"{text}\n\n"
        print()
    return output_text

def text_processing(texts, similarity_thresh, min_words, max_words):
    vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(min_words, max_words)) # setup TFIDF
    X = vectorizer.fit_transform([texts]) # apply TFIDF to text
    words = vectorizer.get_feature_names_out() # words/phrases from TFIDF
    scores = np.asarray(X.mean(axis=0))[0] # scores for above words/phrases from TFIDF
    embeddings = model.encode(words) # embed words into latent space
    normed = normalize(embeddings) # normalize embeddings
    clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=similarity_thresh) # set up clustering
    clustering.fit(normed) # cluster words into groups
    labels = clustering.labels_
    # combine words and scores from clusters
    df = None
    for n in range(clustering.n_clusters_): # for each cluster
        mask = (labels == n) # get all matching words in cluster
        text, summ = "", 0
        for word, score in zip(words[mask], scores[mask]): # for each word and score in cluster
            text += f"{word} " # smoosh
            summ += score # combine scores in cluster
        text = "/".join(set(text.split(" ")))
        if df is None:
            df = pd.DataFrame({"Idea":[text], "Score":[summ]}) # create dataframe
        else:
            df = pd.concat([df, pd.DataFrame({"Idea":[text], "Score":[summ]})]) # append to dataframe

    df = df.round(3)
    df.sort_values(by="Score", ascending=False) # sort dataframe by score
    return df

def save_to_csv(df:pd.DataFrame):
    print(os.getcwd()+"output.csv")
    df.to_csv(os.getcwd()+"/output.csv")


with gr.Blocks() as demo:
    # "global" variables
    # embeddings = gr.State(None)
    pipe = pipeline("image-to-text", model="microsoft/trocr-base-handwritten")
    model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

    with gr.Column() as outer_col:
        with gr.Row() as row1:
            grouping_strength = gr.Number(value=1.2, label="phrase grouping strength", maximum=2, minimum=0, precision=2, step=0.01)
            min_words = gr.Number(value=3, label="phrase length minimum", precision=0, minimum=1)
            max_words = gr.Number(value=5, label="phrase length maximum", precision=0, minimum=1)
        with gr.Row() as row2:
            with gr.Column() as col1:
                files = gr.Files(label="images", file_types=["image"])
                process_button1 = gr.Button(value="Process")
                text_output = gr.Textbox(label="")
                process_button1.click(fn=image_to_text, inputs=[files], outputs=[text_output])
            with gr.Column() as col2:
                textbox = gr.Textbox(label="text to condense")
                process_button2 = gr.Button(value="Process")
                df = gr.DataFrame(wrap=True)
                process_button2.click(fn=text_processing, inputs=[textbox, grouping_strength, min_words, max_words], outputs=[df])
                save_to_csv_button = gr.Button(value="Save To CSV")
                save_to_csv_button.click(fn=save_to_csv, inputs=df)
demo.launch()

/Users/isaac/Desktop/Creative/Coding/momApp/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/private/var/folders/q7/brbj2bqs68lgmt6f_sklpqfh0000gn/T/gradio/bd6ec13cdb2a6065f04e689a929ee82b74aeb9e1/Screenshot 2024-04-21 at 5.09.43PM.png
/private/var/folders/q7/brbj2bqs68lgmt6f_sklpqfh0000gn/T/gradio/102b8f6398445d022676540c7fbd14c2de0815f7/Screenshot 2024-04-21 at 5.09.43PM copy.png
5


/Users/isaac/Desktop/Creative/Coding/momApp/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [157]:
demo.close()

Closing server running on port: 7860
